#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 5, 9)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,15,S,SPALI,"3,000",22.40,1,C22.0,SET100,2022-04-27


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index()

trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
B,DCC,-1,ROUND,SET,30000,2.940000,2.960000,-0.040000,-1.33%,0,2022-05-13,Buy,88200.000000
B,DOHOME,-2,ROUND,SET100,2400,19.500000,19.700000,-0.400000,-1.99%,0,2022-02-02,Buy,46800.000000
B,JASIF,-2,ROUND,SET,10000,10.800000,11.000000,0.000000,0.00%,0,2022-03-03,Buy,108000.000000
B,SENA,-3,RD05pct,SET,15000,4.280000,4.340000,-0.040000,-0.91%,0,2022-05-10,Buy,64200.000000
B,BCH,-13,RD05pct,SET100,3000,19.900000,21.200000,0.000000,0.00%,0,2022-05-10,Buy,59700.000000


### End of Tables in the process

### Print to verify before upload file

In [16]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [17]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [18]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,S,MCS,"5,000",13.10,"65,500.00",C12.80,SET,2,2022-03-15
1,S,PTT,"3,000",37.75,"113,250.00",C38.50,SET50,2,2022-03-03
2,S,RATCH,"2,000",42.00,"84,000.00",CUT,SET50,2,2022-03-17


In [19]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
S    262750.0
Name: amount, dtype: float64

In [20]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate


In [21]:
cash = 20_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(20000, 0.0, 20000.0)

In [22]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,S,MCS,"5,000",13.10,"65,500.00",C12.80,SET,2,2022-03-15
1,S,PTT,"3,000",37.75,"113,250.00",C38.50,SET50,2,2022-03-03
2,S,RATCH,"2,000",42.00,"84,000.00",CUT,SET50,2,2022-03-17


In [23]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(20000, 262750.0, 282750.0)

### After call ord-log (must call everytime that orders change)

In [ ]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [ ]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

In [ ]:
mb = (df.trans == 'Buy') & (df.spd >= -3)
df[mb].sort_values(['name'],ascending=[True])

In [ ]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

In [ ]:
ms = (df.trans == 'Sell') & (df.spd <= 3)
df[ms].sort_values(['name'],ascending=[True])

In [ ]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

In [ ]:
df[mb | ms].shape[0]

In [ ]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

### Select source of orders between these two

In [ ]:
df_out = df[mb | ms][colt]
df_out

In [ ]:
df_out = df[df.active == 2][colt]
df_out

In [ ]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process